```shell
pip install flask-wtf
```

In [1]:
from IPython.core.display import display, HTML

def show(string):
    display(HTML(string))

# a. Configuration

In [2]:
from flask import Flask, render_template
from flask_bootstrap import Bootstrap
from flask_moment import Moment

app = Flask(__name__)
bootstrap = Bootstrap(app)
moment = Moment(app)



app.config['SECRET_KEY'] = 'hard to guess string'

app.config['SERVER_NAME'] = 'floydjjluo.me'
#app.config['SERVER_NAME'] = ''
app.config['WTF_CSRF_ENABLED'] = False

# b. Form Classes

In [3]:
from flask import Flask, render_template, session, redirect, url_for, flash
from flask_bootstrap import Bootstrap
from flask_moment import Moment

# new
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField
from wtforms.validators import DataRequired

In [4]:
FlaskForm

flask_wtf.form.FlaskForm

In [5]:
class NameForm(FlaskForm):
    name = StringField('What is your name?', validators=[DataRequired()])
    submit = SubmitField('Submit')

NameForm.name

<UnboundField(StringField, ('What is your name?',), {'validators': [<wtforms.validators.DataRequired object at 0x10bd4d0f0>]})>

# c. HTML Rendering of Forms

In [6]:
%%writefile templates/c_forms.html

<form method="POST">
    {{ form.hidden_tag() }}
    {{ form.name.label }} {{ form.name() }}
    {{ form.submit() }}
</form>


<form method="POST">
    {{ form.hidden_tag() }}
    {{ form.name.label }} {{ form.name(id = 'my-text-filed') }}
    {{ form.submit() }}
</form>

<p>Hello, your name is {{name}}</p>



Overwriting templates/c_forms.html


In [7]:
@app.route('/c_forms', methods=['GET', 'POST'])
def c_forms():
    name = None
    form = NameForm()
    if form.validate_on_submit():
        name = form.name.data
        form.name.data = ''
    o = render_template('c_forms.html', form=form, name=name); show(o)
    return o

---

** The problem can be resolved by Testing **


The FLAKS TEST CLIENT


---

In [8]:
app.config['WTF_CSRF_ENABLED'] = False

In [9]:
app_ctx = app.app_context()
app_ctx.push()
client = app.test_client(use_cookies=True)

In [10]:
resp = client.get('/c_forms')

In [11]:
resp = client.post('/c_forms', data = {'name': 'Floyd'} )

In [12]:
resp.get_data(as_text = True)

'\n<form method="POST">\n    \n    <label for="name">What is your name?</label> <input id="name" name="name" type="text" value="">\n    <input id="submit" name="submit" type="submit" value="Submit">\n</form>\n\n\n<form method="POST">\n    \n    <label for="name">What is your name?</label> <input id="my-text-filed" name="name" type="text" value="">\n    <input id="submit" name="submit" type="submit" value="Submit">\n</form>\n\n<p>Hello, your name is Floyd</p>'

In [13]:
app_ctx.pop()

# d. Form Handling in View Functions

In [14]:
%%writefile templates/d_index.html

{% extends "base.html" %}
{% import "bootstrap/wtf.html" as wtf %}

{% block title %}Flasky{% endblock %}

{% block page_content %}
<div class="page-header">
    <h1>Hello, {% if name %}{{ name }}{% else %}Stranger{% endif %}!</h1>
</div>
{{ wtf.quick_form(form) }}
{% endblock %}



Overwriting templates/d_index.html


In [15]:
@app.route('/d_index', methods=['GET', 'POST'])
def d_index():
    name = None
    form = NameForm()
    if form.validate_on_submit():
        name = form.name.data
        form.name.data = ''
    o = render_template('index.html', form=form, name=name); show(o)
    return o


In [16]:
app_ctx = app.app_context()
app_ctx.push()
client = app.test_client(use_cookies=True)

In [17]:
resp = client.get('/d_index')

In [18]:
resp = client.post('/d_index', data ={'name' : 'Floyd'})

# e. Redirects and User Sessions

In [19]:
app_ctx = app.app_context()
app_ctx.push()
client = app.test_client(use_cookies=True)

In [20]:
from flask import Flask, render_template, session, redirect, url_for

@app.route('/e_index', methods=['GET', 'POST'])
def e_index():
    form = NameForm()
    if form.validate_on_submit():
        session['name'] = form.name.data
        # return redirect(url_for('e_index'))
        return e_index()
    o = render_template('index.html', form=form, name=session.get('name')); show(o)
    return o


In [21]:
url_for('e_index')

'http://floydjjluo.me/e_index'

In [22]:
redirect('http://floydjjluo.me/e_index')

<Response 263 bytes [302 FOUND]>

In [23]:
client.get('/e_index')

<Response streamed [200 OK]>

In [24]:
client.post('/e_index')

<Response streamed [200 OK]>

In [25]:
client.post('/e_index', data = {'name': 'Floyd'})
# please kindly note that the form is in red

<Response streamed [200 OK]>

In [26]:
client.get('/e_index') # session remembered me

<Response streamed [200 OK]>

# f. Message Flashing

In [27]:
app_ctx = app.app_context()
app_ctx.push()
client = app.test_client(use_cookies=True)

In [28]:
%%writefile templates/f_base_flashing.html

{% extends "bootstrap/base.html" %}

{% block title %}Flasky{% endblock %}

{% block head %}
{{ super() }}
<link rel="shortcut icon" href="{{ url_for('static', filename='favicon.ico') }}" type="image/x-icon">
<link rel="icon" href="{{ url_for('static', filename='favicon.ico') }}" type="image/x-icon">
{% endblock %}

{% block navbar %}
<div class="navbar navbar-inverse" role="navigation">
    <div class="container">
        <div class="navbar-header">
            <button type="button" class="navbar-toggle" data-toggle="collapse" data-target=".navbar-collapse">
                <span class="sr-only">Toggle navigation</span>
                <span class="icon-bar"></span>
                <span class="icon-bar"></span>
                <span class="icon-bar"></span>
            </button>
            <a class="navbar-brand" href="/">Flasky</a>
        </div>
        <div class="navbar-collapse collapse">
            <ul class="nav navbar-nav">
                <li><a href="/">Home</a></li>
            </ul>
        </div>
    </div>
</div>
{% endblock %}

<!-----------------NEW------------------------>
{% block content %}
<div class="container">
    {% for message in get_flashed_messages() %}
    <div class="alert alert-warning">
        <button type="button" class="close" data-dismiss="alert">&times;</button>
        {{ message }}
    </div>
    {% endfor %}

    {% block page_content %}{% endblock %}
</div>
{% endblock %}
<!-----------------NEW------------------------>



{% block scripts %}
{{ super() }}
{{ moment.include_moment() }}
{% endblock %}


Overwriting templates/f_base_flashing.html


In [29]:
%%writefile templates/f_flash.html

{% extends "f_base_flashing.html" %}
{% import "bootstrap/wtf.html" as wtf %}

{% block title %}Flasky{% endblock %}

{% block page_content %}
<div class="page-header">
    <h1>Hello, {% if name %}{{ name }}{% else %}Stranger{% endif %} !</h1>
</div>
{{ wtf.quick_form(form) }}
{% endblock %}


Overwriting templates/f_flash.html


In [30]:
from flask import Flask, render_template, session, redirect, url_for, flash

@app.route('/f_flash', methods=['GET', 'POST'])
def f_flash():
    
    form = NameForm()
    
    if form.validate_on_submit():
        old_name = session.get('name')
        if old_name and old_name != form.name.data:
            flash('This is a flashing, which appears if you change the name!')
        session['name'] = form.name.data
        # return redirect(url_for('f_flash'))
        return f_flash()
    
    o = render_template('f_flash.html', 
                         form = form, 
                         name = session.get('name')); show(o)
    return o



In [31]:
client.get('/f_flash')

<Response streamed [200 OK]>

In [32]:
client.post('/f_flash')

<Response streamed [200 OK]>

In [34]:
client.post('/f_flash', data = {'name': 'Floyd'}) # note the red rectangular

<Response streamed [200 OK]>

In [36]:
client.get('/f_flash') 

<Response streamed [200 OK]>

In [37]:
client.post('/f_flash', data = {'name': 'JieJieJie'}) 

<Response streamed [200 OK]>

In [38]:
client.get('/f_flash') 

<Response streamed [200 OK]>